In [ ]:
"""
Joinability Analysis using MinHash and Locality Sensitive Hashing (LSH)

This notebook demonstrates the use of MinHash and LSH for identifying joinable columns across multiple datasets. 
The analysis is performed on a collection of CSV files stored in the './somervillema' directory.

Overview of the Process:
1. Read each CSV file in the specified directory.
2. For each column in these CSV files, create a MinHash signature. 
3. Insert these MinHash signatures into an LSH index.
4. Perform a query using a specific column's MinHash to find similar columns in the collection of datasets.
5. Display the results, showing which columns from different datasets are potentially joinable with the query column.

Functions:
- create_minhash(column): Generates a MinHash signature for a given column from a DataFrame.

Key Variables:
- lsh: An instance of MinHashLSH, used for indexing MinHash signatures.
- minhashes: A dictionary that stores the MinHash signatures of each column.
- query_key: A tuple representing the dataset and column name used for querying the LSH index.

Query Execution:
- The query is executed for the 'Year' column in the '3ms3-ngki.csv' dataset.
- The LSH index is queried to find columns similar to this 'Year' column.
- Results are filtered to exclude the query column itself and then printed.

Note: The script handles EmptyDataError to skip over any CSV files that are empty or unreadable.
"""

In [2]:
import pandas as pd
from datasketch import MinHash, MinHashLSH
import os

In [3]:
def create_minhash(column):
    m = MinHash()
    for val in column:
        m.update(str(val).encode('utf8'))
    return m

In [76]:
total_column_counts = {}

lsh = MinHashLSH(threshold=0.2, num_perm=128)
minhashes={}
for filename in os.listdir('./somervillema'):
    if filename.endswith('.csv'):
        try:
            df = pd.read_csv(f'./somervillema/{filename}',header=0,dtype=str)
            df = df.dropna(axis=1, how='all')
            total_column_counts[filename] = len(df.columns)
        except pd.errors.EmptyDataError:
            continue
        for column in df.columns:
            mh = create_minhash(df[column])
            minhashes[(filename,column)] = mh
            lsh.insert(f"{filename}_{column}",mh)


In [77]:
query_key = ('3ms3-ngki.csv', 'Year')

if query_key in minhashes:
    # Query the LSH for similar columns
    result = lsh.query(minhashes[query_key])

    # Filter out any column from the same dataset as the query_key
    filtered_result = [r for r in result if not r.startswith(f"{query_key[0]}_")]

    # Print results
    print(f"Columns similar to {query_key}:")
    for res in filtered_result:
        print(res)
else:
    print(f"No MinHash found for {query_key}")

Columns similar to ('3ms3-ngki.csv', 'Year'):
3qxw-3aiy.csv_mmwr_year
6j4n-batb.csv_Year
bi8e-5vw8.csv_Year
qu9x-4xq5.csv_Year


In [78]:
# for key in minhashes:
#     print(key,minhashes[key])

In [79]:
query_key = ('3ms3-ngki.csv', 'Latitude')
# Check if the query key exists in the minhashes dictionary
if query_key in minhashes:
    # Query the LSH for similar columns
    result = lsh.query(minhashes[query_key])

    # Filter out the query_key itself from the results
    filtered_result = [r for r in result if not r.startswith(f"{query_key[0]}_")]

    # Print results
    print(f"Columns similar to {query_key}:")
    for res in filtered_result:
        print(res)
else:
    print(f"No MinHash found for {query_key}")

Columns similar to ('3ms3-ngki.csv', 'Latitude'):
6j4n-batb.csv_Latitude


In [80]:
# Calculating Similarity and Joinability Score for each tabular dataset in Somerville

# Dictionaries for counts
similar_column_counts = {}
joinable_column_sets = {dataset: set() for dataset in total_column_counts}

# Loop through every column in every dataset
for key in minhashes:
    dataset, column = key
    
    result = lsh.query(minhashes[key])
    filtered_result = [r for r in result if r.split('_')[0] != dataset]

    # Update similar column counts and joinable column sets
    if filtered_result:
        joinable_column_sets[dataset].add(column)
    for res in filtered_result:
        res_dataset = res.split('_')[0]
        similar_column_counts[res_dataset] = similar_column_counts.get(res_dataset, 0) + 1

# Calculate and print the similarity and joinability score for each dataset
print("\nSimilarity Scores:")
for dataset in total_column_counts:
    similar_count = similar_column_counts.get(dataset, 0)
    total_count = total_column_counts[dataset]
    joinable_count = len(joinable_column_sets[dataset])
    similarity_score = similar_count / total_count if total_count > 0 else 0
    joinability_score = joinable_count / total_count if total_count > 0 else 0
    print(f"{dataset}: Similarity Score = {similarity_score:.2f}\t Joinability Score = {joinability_score:.2f}")


Similarity Scores:
3qxw-3aiy.csv: Similarity Score = 6.89	 Joinability Score = 0.33
ezmv-8wys.csv: Similarity Score = 47.84	 Joinability Score = 0.89
cmth-mghs.csv: Similarity Score = 16.47	 Joinability Score = 0.67
qsv6-v7hu.csv: Similarity Score = 29.85	 Joinability Score = 0.65
6j4n-batb.csv: Similarity Score = 13.57	 Joinability Score = 1.00
4pyi-uqq6.csv: Similarity Score = 12.00	 Joinability Score = 0.20
8r94-vs2v.csv: Similarity Score = 1.00	 Joinability Score = 0.33
qu9x-4xq5.csv: Similarity Score = 32.14	 Joinability Score = 1.00
wz6k-gm5k.csv: Similarity Score = 1.47	 Joinability Score = 0.21
vxgw-vmky.csv: Similarity Score = 1.00	 Joinability Score = 0.30
9wbi-ck3z.csv: Similarity Score = 3.00	 Joinability Score = 0.43
754v-8e35.csv: Similarity Score = 1.40	 Joinability Score = 0.40
bi8e-5vw8.csv: Similarity Score = 36.51	 Joinability Score = 0.82
xavb-4s9w.csv: Similarity Score = 13.70	 Joinability Score = 0.60
62z4-avqc.csv: Similarity Score = 9.14	 Joinability Score = 0.